In [11]:
from  langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.document_loaders import TextLoader
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


loader = TextLoader("notes.txt", encoding="utf-8")
docs = loader.load()
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size =100, chunk_overlap = 0)
chunks = text_splitter.split_documents(docs)
embeded = HuggingFaceEmbeddings(model_name= "all-MiniLM-L6-V2")
vectorstore = FAISS.from_documents(chunks, embeded)
retriever = vectorstore.as_retriever()

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Step 2: Create the pipeline (text2text-generation, not text-generation)
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100,
    temperature=0.7,
    do_sample=True,
)

# Use in LangChain
llm = HuggingFacePipeline(pipeline=pipe)
qa = RetrievalQA.from_chain_type(llm=llm, retriever = retriever)
def self_rag_query(Question):
    print("first attempt without retriever:")
    first_answer = llm.predict(f"Q:{Question}\nA:")
    if "Iam not sure"in first_answer or len(first_answer) <30:
        print("low confidence Retrieving context and trying again...")
        improved_answer = qa.run(Question)
        return improved_answer
    else:
        return first_answer
response = self_rag_query("principle of GEN AI?")
print("\nFinal answer:", response)
 



Created a chunk of size 292, which is longer than the specified 100
Created a chunk of size 351, which is longer than the specified 100
Created a chunk of size 117, which is longer than the specified 100
Device set to use cpu


first attempt without retriever:


Token indices sequence length is longer than the specified maximum sequence length for this model (1144 > 512). Running this sequence through the model will result in indexing errors


low confidence Retrieving context and trying again...

Final answer: of supervised learning: Principles of Generative AI A Technical Introduction Generative AI tools are an emerging class of new-age artificial intelligence algorithms capable of producing novel content . supervised learning is an emerging class of new-age artificial intelligence algorithms capable of producing novel content — in varied formats such as text, audio, video, pictures, and code . this report focuses almost exclusively on large language models (LLMs) capable of generating novel content 
